In [ ]:
from openai import OpenAI, BadRequestError
from openai.types.chat import ChatCompletion
from tenacity import retry, stop_after_attempt, wait_random_exponential
import time
from typing import Optional
import base64
import requests
import json
import os
import re
from tqdm import tqdm
import yaml
from openai import OpenAI
import dataclasses
import textwrap
import pandas as pd
import pickle
from pathlib import Path
import pprint
from collections import defaultdict


In [2]:
class MinimumDelay:
    def __init__(self, delay: float | int):
        self.delay = delay
        self.start = None

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        end = time.time()
        seconds = end - self.start
        if self.delay > seconds:
            time.sleep(self.delay - seconds)

@retry(wait=wait_random_exponential(min=1, max=90), stop=stop_after_attempt(3))
def chat(client: OpenAI, delay: float | int, **kwargs) -> ChatCompletion | None:
    try:
        with MinimumDelay(delay):
            return client.chat.completions.create(**kwargs)
    except BadRequestError as e:
        print(f"Bad Request: {e}")
        if "safety" in e.message:
            return None
        raise e
    except Exception as e:
        print(f"Exception: {e}")
        raise e
def print_messages(messages):
    for message in messages:
        if isinstance(message["content"], list):
            print(f"{message['role']}:")
            for content in message["content"]:
                if content["type"] == "text":
                    print(content["text"])
                elif content["type"] == "image_url":
                    print("[IMAGE]")
        else:
            print(f"{message['role']}: {message['content']}")
        print()
    print("=========================================")

def read_jsonl(path):
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                ex = json.loads(line)
                yield ex

def write_jsonl(path, data):
    with open(path, "w") as f:
        for ex in data:
            f.write(json.dumps(ex) + "\n")

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

Step 1: HTD

In [3]:
from pydantic import BaseModel, Field
import instructor

client = instructor.from_openai(OpenAI(api_key=os.environ['OPENAI_API_KEY'], timeout= 90))
class InstructorChatCompletionConfig(BaseModel):
    seed: int = Field(..., description="Random seed for reproducibility")
    delay: int = Field(..., description="Minimum delay between requests in seconds")
    model: str = Field(..., description="Model to use for chat completion")
    max_tokens: int = Field(..., description="Maximum number of tokens in the response")
    temperature: float = Field(..., description="Sampling temperature for response generation")
    system_prompt: str = Field(..., description="System prompt to set the context for the conversation")
    user_prompt: str = Field(..., description="User prompt to initiate the conversation")
    response_format: dict | None = Field(None, description="Expected format of the response")

class Feedback(BaseModel):
    feedback: list[str] = Field(
        description="A list of actions to take to make predicted values more consistent with the value taxonomy."
    )
    is_consistent: bool



In [ ]:
from pydantic import BaseModel, Field
from typing import List


class Level1Value(BaseModel):
    level_1_value: str = Field(
        description="Level 1 Shwartz cultural value applicable to the frame.",
        enum=["Openness to Change", "Self-Enhancement", "Conservation", "Self-Transcendence"]
    )
    rationale: str = Field(description="Rationale explaining why the level 1 value is applicable to the frame.")


class Level2Value(BaseModel):
    level_2_value: str = Field(
        description="Level 2 cultural value applicable to the frame.",
        enum=[
            "Self-direction (thought)",
            "Self-direction (action)",
            "Stimulation",
            "Hedonism",
            "Achievement",
            "Power (dominance)",
            "Power (resources)",
            "Face",
            "Security (personal)",
            "Security (societal)",
            "Tradition",
            "Conformity (rules)",
            "Conformity (interpersonal)",
            "Humility",
            "Benevolence (caring)",
            "Benevolence (dependability)",
            "Universalism (concern)",
            "Universalism (nature)",
            "Universalism (tolerance)",
            "Universalism (objectivity)"
        ]
    )
    rationale: str = Field(description="Rationale explaining why the level 2 value is applicable to the frame.")


class Level3Value(BaseModel):
    level_3_value: str = Field(description="Level 3 cultural value applicable to the frame.", 
            enum=[
                "Be creative",
                "Be curious",
                "Have freedom of thought",
                "Be choosing own goals",
                "Be independent",
                "Have freedom of action",
                "Have privacy",
                "Have an exciting life",
                "Have a varied life",
                "Be daring",
                "Have pleasure",
                "Be ambitious",
                "Have success",
                "Be capable",
                "Be intellectual",
                "Be courageous",
                "Have influence",
                "Have the right to command",
                "Have wealth",
                "Have social recognition",
                "Have a good reputation",
                "Have a sense of belonging",
                "Have good health",
                "Have no debts",
                "Be neat and tidy",
                "Have a comfortable life",
                "Have a safe country",
                "Have a stable society",
                "Be respecting traditions",
                "Be holding religious faith",
                "Be compliant",
                "Be self-disciplined",
                "Be behaving properly",
                "Be polite",
                "Be honoring elders",
                "Be humble",
                "Have life accepted as is",
                "Be helpful",
                "Be honest",
                "Be forgiving",
                "Have the own family secured",
                "Be loving",
                "Be responsible",
                "Have loyalty towards friends",
                "Have equality",
                "Be just",
                "Have a world at peace",
                "Be protecting the environment",
                "Have harmony with nature",
                "Have a world of beauty",
                "Be broadminded",
                "Have the wisdom to accept others",
                "Be logical",
                "Have an objective view"
            ])
    rationale: str = Field(description="Rationale explaining why the level 3 value is applicable to the frame.")


class Values(BaseModel):
    level_1_values: List[Level1Value] = Field(..., alias="level 1 values")
    level_2_values: List[Level2Value] = Field(..., alias="level 2 values")
    level_3_values: List[Level3Value] = Field(..., alias="level 3 values")


class CulturalValues(BaseModel):
    values: Values

class Timestep(BaseModel):
    response: CulturalValues
    feedback: Optional[list[str]] = Field(default_factory=list)
    refined_response: Optional[CulturalValues]


class History(BaseModel):
    history: list[Timestep] = Field(default_factory=list)

    def add(self, code, feedback, refined_code):
        self.history.append(
            Timestep(response=code, feedback=feedback, refined_response=refined_code)
        )
def stop_condition(feedback, history):
    return feedback.is_consistent or len(history.history) >= 3 

In [5]:
config_file_path = 'prompts/hierarchical2.yaml'
with open(config_file_path, 'r') as f:
    config = yaml.safe_load(f)
    config = InstructorChatCompletionConfig(**config)

with open('prompts/feedback.yaml', 'r') as f:
    feedback_config = yaml.safe_load(f)
    feedback_config = InstructorChatCompletionConfig(**feedback_config)

with open('prompts/refine.yaml', 'r') as f:
    refine_config = yaml.safe_load(f)
    refine_config = InstructorChatCompletionConfig(**refine_config)


In [6]:
def get_valid_level2_actions(level1_action: str) -> list[str]:
    """
    Get valid level 2 actions based on the level 1 action.
    """
    if level1_action == "Openness to Change":
        return["Self-direction (thought)",
                      "Self-direction (action)",
                      "Stimulation",
                      "Hedonism"]   
                    
    elif level1_action == "Self-Enhancement":
        return ["Hedonism", "Achievement",
                      "Power (dominance)",
                      "Power (resources)", "Face"]
    elif level1_action == "Conservation":
        return ["Face","Security (personal)",
                      "Security (societal)",
                      "Tradition",
                      "Conformity (rules)",  
                      "Conformity (interpersonal)",
                      "Humility"]
    elif level1_action == "Self-Transcendence":
        return ["Humility", "Benevolence (caring)",
                "Benevolence (dependability)",
                      "Universalism (concern)",
                      "Universalism (nature)",
                      "Universalism (tolerance)"
                      "Universalism (objectivity)"]
    else:
        return []
    
def get_valid_level3_actions(level2_action: str) -> list[str]:
    """
    Get valid level 3 actions based on the level 2 action.
    """
    if level2_action == "Self-direction (thought)":
        return ["Be creative",
                "Be curious",
                "Have freedom of thought",]
    elif level2_action == "Self-direction (action)":
        return ["Be choosing your own goals",
                "Be independent",
                "Have freedom of action","Have privacy"]
    elif level2_action == "Stimulation":
        return ["Have an exciting life",
                "Have a varied life",
                "Be daring",]
    elif level2_action == "Hedonism":
        return ["Have pleasure"]
    elif level2_action == "Achievement":
        return ["Be ambitious",
                "Have success"
                "Be capable",
                "Be intellectual",
                "Be courageous"]
    elif level2_action == "Power (dominance)":
        return ["Have influence",
                "Have the right to command"]
    elif level2_action == "Power (resources)":
        return ["Have wealth"]
    elif level2_action == "Face":
        return ["Have a social recognition",
                "Have a good reputation",
        ]
    elif level2_action == "Security (personal)":
        return ["Have a sense of belonging",
                "Have good health",
                "Have no debts",
                "Be neat and tidy",
                "Have a comfortable life"]
    elif level2_action == "Security (societal)":
        return ["Have a safe country",
                "Have a stable society",]
    elif level2_action == "Tradition":
        return ["Be respecting traditions",
                "Be holding religious faith"]
    elif level2_action == "Conformity (rules)":
        return ["Be compliant",
                "Be self-disciplined",
                "Be behaving properly",]
    elif level2_action == "Conformity (interpersonal)":
        return ["Be polite",
                "Be honouring elders",]
    elif level2_action == "Humility":
        return ["Be humble",
                "Have life accepted as is"]
    elif level2_action == "Benevolence (caring)":
        return ["Be helpful",
                "Be honest",
                "Be forgiving",
                "Have the own family secured",
                "Be loving"]
    elif level2_action == "Benevolence (dependability)":
        return ["Be responsible",
                "Have loyalty towards friends"]
    elif level2_action == "Universalism (concern)":
        return ["Have equality",
                "Be just",
                "Have a world at peace"]
    elif level2_action == "Universalism (nature)":
        return ["Be protecting the environment",
                "Have harmony with nature",
                "Have a world of beauty"]
    elif level2_action == "Universalism (tolerance)":
        return ["Be broadminded",
                "Have the wisdom to accept others"]
    
    elif level2_action == "Universalism (objectivity)":
        return ["Be logical",
                "Have an objective view"]
    else:
        return []
            
    

In [7]:
def generate_response(config: InstructorChatCompletionConfig, frame: str, prob_def: str, level1: str, level2:str, level3:str):
    with open("cultural_values_taxonomy.yaml", "r") as f:
        values_yaml = f.read()

    sys_prompt = config.system_prompt.strip()
    user_prompt= config.user_prompt.strip()
    final_sys_prompt=sys_prompt.format(values_yaml=values_yaml)
    msg=[{"role": "system", "content": final_sys_prompt},
         {"role": "user", "content": user_prompt.format(Frame=frame, Definition_str=prob_def, level_1_values = level1, level_2_values= level2,
                                                        level_3_values=level3)}]
    #print_messages(msg)
    return client.chat.completions.create(
        model=config.model,
        messages=msg,
        temperature=config.temperature,
        response_model=CulturalValues,
        seed=config.seed
    )

def get_levels_from_response(response: dict) -> dict:
    level1_vals=[]
    level1_rats=[]
    level2_vals=[]
    level2_rats=[]
    level3_vals=[]
    level3_rats=[]
    for l in response.values.level_1_values:
        level1_vals.append(l.level_1_value)
        level1_rats.append(l.rationale)
    for l in response.values.level_2_values:
        level2_vals.append(l.level_2_value)
        level2_rats.append(l.rationale)
    for l in response.values.level_3_values:
        level3_vals.append(l.level_3_value)
        level3_rats.append(l.rationale)

    app_level2_vals = []
    seen = set()

    for val in level1_vals:
        for lvl2 in get_valid_level2_actions(val):
            if lvl2 not in seen:
                seen.add(lvl2)
                app_level2_vals.append(lvl2)

    app_level3_vals = []
    seen = set()
    for val in level2_vals:
        for lvl3 in get_valid_level3_actions(val):
            if lvl3 not in seen:
                app_level3_vals.append(lvl3)
                
    return {"level1_vals": level1_vals, "level1_rats": level1_rats, "level2_vals": level2_vals, "level2_rats": level2_rats, 
            "level3_vals": level3_vals, "level3_rats": level3_rats, "app_level2_vals": app_level2_vals, "app_level3_vals": app_level3_vals}



def generate_feedback(config:InstructorChatCompletionConfig, response: dict, frame: str):
    sys_prompt = config.system_prompt.strip()
    user_prompt = config.user_prompt.strip()

    response_data = get_levels_from_response(response)

    

    final_usr= user_prompt.format(
        Frame=frame,
        level_1_values=response_data['level1_vals'],
        level_1_rationale=response_data['level1_rats'],
        level_2_values=response_data['level2_vals'],
        level_2_rationale=response_data['level2_rats'],
        level_3_values=response_data['level3_vals'],
        level_3_rationale=response_data['level3_rats'],
        app_level_2_values=response_data['app_level2_vals'],
        app_level_3_values= response_data['app_level3_vals'],
    )
    msg = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": final_usr},
    ]
    #print_messages(msg)
    return client.chat.completions.create(
        model=config.model,
        messages=msg,
        temperature=config.temperature,
        response_model=Feedback,
        seed=config.seed
    )

def refine_response(config: InstructorChatCompletionConfig, response: Feedback, response0: dict, frame: str):

    sys_prompt = config.system_prompt.strip()
    user_prompt = config.user_prompt.strip()
    response_data = get_levels_from_response(response0)
    fdbk_str = "\n".join(response.feedback)
    final_usr= user_prompt.format(Frame=frame, level_1_values=response_data['level1_vals'], level_2_values=response_data['level2_vals'], level_3_values=response_data['level3_vals'],
                                  feedback=fdbk_str)
    msg = [{"role": "system", "content": sys_prompt},
              {"role": "user", "content": final_usr}]
    #print_messages(msg)
    return client.chat.completions.create(
        model=config.model,
        messages=msg,
        temperature=config.temperature,
        response_model=CulturalValues,
        seed=config.seed
    )
    

In [1]:
import os
import json
base_dir = "final_frames"  # path to your folder

for problem_folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, problem_folder)
    values_file = os.path.join(folder_path, "values.jsonl")
    
    if os.path.isdir(folder_path) and os.path.exists(values_file):
        value_dict = {}

        with open(values_file, "r", encoding="utf-8") as f:
            for line in f:
                entry = json.loads(line)
                value_dict[entry["id"]] = entry["response"]

        # Write the dictionary to value_dict.json
        output_file = os.path.join(folder_path, "value_dict.json")
        with open(output_file, "w", encoding="utf-8") as out:
            json.dump(value_dict, out, indent=2, ensure_ascii=False)

In [8]:
def extract_values(entry):
    values = entry["values"]
    
    level_3_values = "\n".join(v["level_1_value"] for v in values.get("level_1_values", []))
    level_2_values = "\n".join(v["level_2_value"] for v in values.get("level_2_values", []))
    level_1_values = "\n".join(v["level_3_value"] for v in values.get("level_3_values", []))
    
    return level_1_values, level_2_values, level_3_values

with open("cleaned_misogyny_problem_definitions.json", "r") as f:
    PROBLEM_DEFINITIONS = json.load(f)
frame_problems_dict = {}

for f in read_jsonl('frame_problem_info.jsonl'):
    frame_problems_dict[f['id']] = {
        "frame": f['frame'],
        "problems": f['problems']
    }

In [9]:
for f in sorted(os.listdir('final_frames')):
    folder_path = os.path.join('final_frames', f)
    
    # Check if it's a folder and contains 'values_contra.jsonl'
    if os.path.isdir(folder_path) and os.path.exists(os.path.join(folder_path, 'values_contra.jsonl')):
        continue 
    val_file=os.path.join('final_frames',f,'value_dict.json')
    
    with open(val_file, "r", encoding="utf-8") as e:
        value_dict = json.load(e)

    ffilename = os.path.join('final_frames', f, 'frames.jsonl')

    problem_values=[]
    for frame in read_jsonl(ffilename):
        idf= frame['id']
        problems = frame_problems_dict[idf]['problems']
        problems_definitions_str = "\n".join(
    f"{problem.replace('_', ' ').title()}: {PROBLEM_DEFINITIONS.get(problem, 'No definition available')}"
    for problem in problems
        
)
        print(problems_definitions_str)
        value_entry = value_dict.get(idf)
        level_1_values, level_2_values, level_3_values = extract_values(value_entry)
        print(level_1_values)
        response = generate_response(config, frame['frame'], problems_definitions_str, level_1_values, level_2_values, level_3_values)
        if response is None:
            print(f"Intial response was None, skipping this frame {idf}.")
            continue
        
        history = History()

        while True:
            feedback = generate_feedback(feedback_config, response, frame)
            if stop_condition(feedback, history):
                print(f"Stopping condition met for frame {idf}.")
                break
            if feedback is None:
                print(f"Feedback response was None, skipping this frame {idf}.")
                break
            refined_response = refine_response(refine_config, feedback, response, frame)
            if refined_response is None:
                print(f"Refined response was None, skipping this frame {idf}.")
                break
            history.add(response, feedback.feedback, refined_response)
            response = refined_response
        problem_values.append({
            "id": idf,
            "frame": frame,
            "response": response.model_dump(),
            "history": [step.model_dump() for step in history.history],
        })
    write_jsonl(os.path.join('final_frames', f, 'values_contra.jsonl'), problem_values) 
    print(f"Done {f} problem") 

Promoting Rape Culture: Perpetuation of attitudes, norms, and practices that normalize, trivialize, or excuse sexual violence and harassment
Be respecting traditions
Be behaving properly
Have the right to command
Stopping condition met for frame f1287.
Promoting Rape Culture: Perpetuation of attitudes, norms, and practices that normalize, trivialize, or excuse sexual violence and harassment
Have the right to command
Be daring
Have pleasure
Stopping condition met for frame f354.
Promoting Rape Culture: Perpetuation of attitudes, norms, and practices that normalize, trivialize, or excuse sexual violence and harassment
Be respecting traditions
Have the right to command
Stopping condition met for frame f5388.
Objectification: The action of degrading women to the status of a mere object
Promoting Rape Culture: Perpetuation of attitudes, norms, and practices that normalize, trivialize, or excuse sexual violence and harassment
Have pleasure
Stopping condition met for frame f5387.
Promoting Ra

In [14]:
problem_temp=[]
problem_temp.append({"id": idf,
            "frame": frame,
            "response": response.model_dump(),
            "history": [step.model_dump() for step in history.history],
        })
write_jsonl(os.path.join('values.jsonl'), problem_temp)

Confidence prediction

In [ ]:
@dataclasses.dataclass
class ChatCompletionConfig:
    seed: int
    delay: int
    model: str
    max_tokens: int
    temperature: float
    system_prompt: str
    user_prompt: str
    response_format: dict | None = None

In [ ]:
def make_user_prompt(frame, value, rationale, user):
    msg=[]
    msg.append({"role":"user","content": user.format(frame=frame, value=value, rationale=rationale)})
    return msg


In [ ]:
results=[]
config_file_path = 'prompts/evaluate.yaml'
with open(config_file_path, 'r') as f:
    config = yaml.safe_load(f)
    config = ChatCompletionConfig(**config)
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'], timeout=90)
sys_prompt = config.system_prompt.strip()
user_prompt= config.user_prompt.strip()
msg=[{"role": "system", "content": sys_prompt}]

In [ ]:

for f in sorted(os.listdir('final_frames')):
    val_file=os.path.join('final_frames',f,'values.jsonl')
    for value in read_jsonl(val_file):
        frame_id = value['id']
        frame= value['frame']
        l=value['response']['values']
        if 'level_1_values' in l:
            for x in l['level_1_values']:
                msg.extend(make_user_prompt(msg, frame, x['level_1_value'], x['rationale'],user_prompt))
                try:
                    completion = client.chat.completions.create(
                        model=config.model,
                        messages=msg,
                        temperature=config.temperature,
                        seed=config.seed,
                        response_format=config.response_format,
                    )
                except Exception as e:
                    print(f"API call failed for {frame_id}: {e}")
                    msg.pop()
                    continue
                msg.pop()
                content = completion.choices[0].message.content
                results.append({'id':frame_id, 'frame': frame, 'value': x['level_1_value'], 'rationale': x['rationale'], 'judgement':content['judge']})
        
        if 'level_2_values' in l:
            for x in l['level_2_values']:
                msg.extend(make_user_prompt(msg, frame, x['level_2_value'], x['rationale'],user_prompt))
                try:
                    completion = client.chat.completions.create(
                        model=config.model,
                        messages=msg,
                        temperature=config.temperature,
                        seed=config.seed,
                        response_format=config.response_format,
                    )
                except Exception as e:
                    print(f"API call failed for {frame_id}: {e}")
                    msg.pop()
                    continue
                msg.pop()
                content = completion.choices[0].message.content
                results.append({'id':frame_id, 'frame': frame, 'value': x['level_2_value'], 'rationale': x['rationale'], 'judgement':content['judge']})
        
        if 'level_3_values' in l:
            for x in l['level_3_values']:
                msg.extend(make_user_prompt(msg, frame, x['level_3_value'], x['rationale'],user_prompt))
                try:
                    completion = client.chat.completions.create(
                        model=config.model,
                        messages=msg,
                        temperature=config.temperature,
                        seed=config.seed,
                        response_format=config.response_format,
                    )
                except Exception as e:
                    print(f"API call failed for {frame_id}: {e}")
                    msg.pop()
                    continue
                msg.pop()
                content = completion.choices[0].message.content
                results.append({'id':frame_id, 'frame': frame, 'value': x['level_3_value'], 'rationale': x['rationale'], 'judgement':content['judge']})

    print('Done with {f}')

In [ ]:
write_jsonl('evaluations.jsonl', results)

Cleaning value_contra..replacing 'response' with response in 'history'

In [2]:
import json
import os

for f in sorted(os.listdir('final_frames')):
    input_file = os.path.join('final_frames',f,'values_contra.jsonl')
    output_file = os.path.join('final_frames',f,'values_contra2.jsonl')

    with open(input_file, "r", encoding="utf-8") as infile, \
        open(output_file, "w", encoding="utf-8") as outfile:

        for line in infile:
            entry = json.loads(line)

            if entry.get("history"):
                # Assuming you want the 'response' from the first history item
                first_history_response = entry["history"][0].get("response")
                if first_history_response:
                    entry["response"] = first_history_response

            # Write updated entry
            outfile.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Updated file written to {output_file}")


Updated file written to final_frames/ableism/values_contra2.jsonl
Updated file written to final_frames/age_related_sexualization/values_contra2.jsonl
Updated file written to final_frames/ageism/values_contra2.jsonl
Updated file written to final_frames/appearance_based_discrimination/values_contra2.jsonl
Updated file written to final_frames/biased_judgement/values_contra2.jsonl
Updated file written to final_frames/classism/values_contra2.jsonl
Updated file written to final_frames/coercion/values_contra2.jsonl
Updated file written to final_frames/commitment_phobia/values_contra2.jsonl
Updated file written to final_frames/conditional_respect/values_contra2.jsonl
Updated file written to final_frames/conspiracy_thinking/values_contra2.jsonl
Updated file written to final_frames/cultural_insensitivity/values_contra2.jsonl
Updated file written to final_frames/deflection/values_contra2.jsonl
Updated file written to final_frames/dehumanization_of_women/values_contra2.jsonl
Updated file written t

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import math

image_folder = 'TRAINING'
sources = ["8624.jpg", "9056.jpg", "1528.jpg", "9043.jpg", "9593.jpg", "9502.jpg", "3469.jpg", "10286.jpg", "370.jpg", "3571.jpg", "3837.jpg", "11057.jpg", "375.jpg", "3959.jpg", "3960.jpg", "6832.jpg", "8073.jpg", "5768.jpg", "13333.jpg", "10532.jpg", "1339.jpg", "3330.jpg", "1015.jpg", "5723.jpg", "523.jpg", "3019.jpg", "1336.jpg", "7779.jpg", "4928.jpg", "1011.jpg", "2666.jpg", "3215.jpg", "3612.jpg", "6920.jpg", "2686.jpg", "8362.jpg", "7655.jpg", "389.jpg", "1434.jpg", "2749.jpg", "5167.jpg", "872.jpg", "11543.jpg", "4505.jpg", "11176.jpg", "9146.jpg", "5277.jpg", "10923.jpg", "6160.jpg", "7398.jpg", "1115.jpg", "11551.jpg", "1272.jpg", "2592.jpg", "4945.jpg", "885.jpg", "4913.jpg", "2231.jpg", "10810.jpg", "8221.jpg", "695.jpg", "9456.jpg", "3588.jpg", "1509.jpg", "10177.jpg", "9134.jpg", "4530.jpg", "10486.jpg", "7314.jpg", "8755.jpg", "5116.jpg", "11208.jpg", "6602.jpg", "3861.jpg", "8193.jpg", "1970.jpg", "3943.jpg", "7782.jpg", "4450.jpg", "8659.jpg", "478.jpg", "7347.jpg", "9770.jpg", "6677.jpg", "1349.jpg", "3105.jpg", "6436.jpg", "5472.jpg", "5665.jpg", "4751.jpg", "5303.jpg", "8693.jpg", "5699.jpg", "1564.jpg", "9296.jpg", "2543.jpg", "2483.jpg", "4784.jpg", "4772.jpg", "5018.jpg", "1464.jpg", "7021.jpg", "10187.jpg", "5606.jpg", "1567.jpg", "9444.jpg", "97.jpg", "1504.jpg", "1376.jpg", "5607.jpg", "14155.jpg", "6297.jpg", "2844.jpg", "5488.jpg", "142.jpg", "9044.jpg", "10998.jpg", "7167.jpg", "4970.jpg", "8581.jpg", "4229.jpg", "10977.jpg", "6381.jpg", "6728.jpg", "10.jpg", "7772.jpg", "6599.jpg", "9253.jpg", "2834.jpg"]
num_images = len(sources)
cols = 4
rows = math.ceil(num_images / cols)
fig, axes = plt.subplots(rows, cols, figsize=(16, rows * 4))

axes = axes.flatten() if num_images > 1 else [axes]

for idx, ax in enumerate(axes):
    if idx < num_images:
        img_path = os.path.join(image_folder, sources[idx])
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.set_title(sources[idx], fontsize=10)
        ax.axis('off')
    else:
        ax.axis('off')  # Hide unused subplots

plt.tight_layout()
plt.show()

In [12]:
len(sources)

129